In [1]:
import h5py
import numpy as np
import pandas as pd 
import scipy

C:\Users\byan\AppData\Local\Temp\ipykernel_12252\2735699808.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def read_data(path_of_data):
  """
  read data seaved in h5py format
  assuming group keys that exists are X,y 
  """
  with h5py.File(path_of_data, "r") as f:
    X,y = f["X"][:],f['y'][:]
  return X,y


def signals_log_sum(signal):
    from scipy.special import logsumexp
    logSum=logsumexp(signal,-2)
    return logSum

def signals_rms(signals):
  rms=np.sqrt(np.mean(signals**2,axis=-2))
  return rms 

def signals_std(signals):
  std = np.std(signals,axis=-2)
  return std

def signals_avg_power(signals):
  power=np.mean(signals,axis=-2)
  return power

def signals_mean_abs(signals):
  mean_abs=np.mean(np.abs(signals),axis=-2)
  return mean_abs

def signals_energy(signals):
  energy=np.mean(np.square(np.abs(signals)),axis=-2)
  return energy
 

def get_features(dataset,normalized=True):
  features=np.concatenate([
                               signals_log_sum(dataset),signals_avg_power(dataset),
                               signals_energy(dataset),signals_rms(dataset) ,
                               signals_mean_abs(dataset),signals_std(dataset)
],axis=-1)
  
  return features 

In [3]:
train_18_X,train_18_y= read_data("data/train.hdf5")
val_18_X,val_18_y=read_data("data/test.hdf5")

In [4]:
print(train_18_X.shape,train_18_y.shape)
print(val_18_X.shape,val_18_y.shape)

(15794, 3840, 18) (15794,)
(1566, 3840, 18) (1566,)


In [5]:
#nans_indicies= np.any(np.isnan(train_18_X),axis=(1,2))
#train_18_X,train_18_y = train_18_X[~nans_indicies],train_18_y[~nans_indicies]
#nans_indicies= np.any(np.isnan(val_18_X),axis=(1,2))
#val_18_X,val_18_y = val_18_X[~nans_indicies],val_18_y[~nans_indicies]

In [5]:
print(train_18_X.shape,train_18_y.shape)
print(val_18_y.shape,val_18_y.shape)

(15794, 3840, 18) (15794,)
(1566,) (1566,)


In [6]:
train_18_X = get_features(train_18_X,)
val_18_X = get_features(val_18_X,)

In [8]:
print(train_18_X.shape,train_18_y.shape)
print(val_18_X.shape,val_18_y.shape)

(15794, 108) (15794,)
(15794, 108) (1566,)


In [7]:
train_features_df = pd.DataFrame(np.hstack((train_18_X,train_18_y[:,np.newaxis])))
test_features_df = pd.DataFrame(np.hstack((val_18_X,val_18_y[:,np.newaxis])))


In [10]:
train_features_df.to_csv('data/features_train.csv')
test_features_df.to_csv('data/features_test.csv')